# Web Parsing

April 2019

In [1]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

### CoinMarketCap

In [2]:
def parse_coinmarketcap():
    """Download CoinMarketCap market snapshot."""
    url = 'https://coinmarketcap.com/all/views/all/'
    response=requests.get(url)
    page=response.text
    soup=BeautifulSoup(page,"lxml")
    tables=soup.find_all("table")
    rows=[row for row in tables[0].find_all('tr')]

    df = pd.read_html(tables[0].prettify())[0]
    df = df.iloc[:,1:6]
    df['Circulating Supply'] = (pd.to_numeric(df['Circulating Supply']. \
                                          apply(lambda x: re.sub(r'[\W+]',"",x)), \
                                          errors='coerce').map('{:,.0f}'.format))
    return df

In [3]:
df = parse_coinmarketcap()
df.head()

(2154, 11)
(2154, 5)


,Name,Symbol,Market Cap,Price,Circulating Supply
0,BTC Bitcoin,BTC,"$88,830,158,723",$5038.18,"17,631,387"
1,ETH Ethereum,ETH,"$17,480,911,232",$165.62,"105,550,449"
2,XRP XRP,XRP,"$14,918,722,220",$0.357388,"41,743,765,071"
3,LTC Litecoin,LTC,"$5,538,708,361",$90.46,"61,225,161"
4,BCH Bitcoin Cash,BCH,"$5,253,355,187",$296.56,"17,714,188"
